#Quantum Chess in Cirq
Quantum Chess is a variant of chess that gives players access to extra moves which allow them to create superposition. All moves are applied to the game state via unitary evolution, allowing players to experience effects like superposition, entanglement, and interference. This project provides a limited implementation of the full Quantum Chess move set, executed on a set of qubits representing squares. Quantum Chess defines REST API, that gives the game the ability to offload the quantum state of the board to an external resource. In this notebook we will explore an implementation of this API, using the code from this project. For more information on how the actual Quantum Chess on Cirq implementation works, including details on qubit mapping and error correction, see the documentation here.

First, we need to install the Quantum Chess package from recirq.

In [ ]:
!pip install git+https://github.com/quantumlib/ReCirq/ -q
import recirq
import recirq.quantum_chess.ascii_board as ab

b = ab.AsciiBoard()
b.reset()
print(b)

The ascii board is a convenience that can be used for testing the project imports.  The Quantum Chess REST API defines the interface, which used by the Quantum Chess Engine to assign an external resource to handle the quantum state of the game. This state encodes only the "occupancy" of each square on the board. Each square is mapped to a single qubit, where the state |1> corresponds to the square being occupied by a piece, and |0> is unoccupied. All piece type information, and rules checking, is handled classically within the Quantum Chess Engine. When implementing the API, we only neeed to use the CirqBoard. The following code shows how to initialize a CirqBoard with a single "piece" in square a1, and introduces a helper function to print out the probability distribution, and some diagnostic information from the instance of CirqBoard.


In [ ]:
from recirq.quantum_chess.quantum_board import CirqBoard
from recirq.quantum_chess.bit_utils import ( bit_to_square, xy_to_bit )
from recirq.quantum_chess.move import to_rank

def probabilities_to_board_str(probabilities):
        """Renders a ASCII diagram showing the board probabilities."""
        s = ''
        s += ' +----------------------------------+\n'
        for y in reversed(range(8)):
            s += str(y + 1) + '| '
            for x in range(8):
                bit = xy_to_bit(x, y)
                prob = str(int(100 * probabilities[bit]))
                if len(prob) <= 2:
                    s += ' '
                if prob == '0':
                    s += '.'
                else:
                    s += prob
                if len(prob) < 2:
                    s += ' '
                s += ' '
            s += ' |\n'
        s += ' +----------------------------------+\n    '
        for x in range(8):
           s += to_rank(x) + '   '
        return s

global_board = CirqBoard(1)

def print_game(board, probs):
    board.print_debug_log()
    print('\n' + probabilities_to_board_str(probs) + "\n\n")

probs = global_board.get_probability_distribution()

print_game(global_board, probs)


#Quantum Chess REST API
The Quantum Chess REST API defines an interface for [REST](https://restfulapi.net/) endpoints that the Quantum Chess Engine can be directed to use when interacting with the quantum state of the game. The API declares an interface for three functions:
* initialize
* do_move
* undo_move

All three endpoints must return a json object with the following values:
* probabilities: an array of 64 floating point numbers representing the probability of each square being occupied. Array indices are mapped to board squares in the same order as bitboard bits, i.e. 0 = a1, 1 = b1, ... 63 = h8.
* empty_bitboard: A bitboard with bits set to 1 for all squares known to be empty, i.e. 0% chance of being occupied.
* full_bitboard: A bitboard with bits set to 1 for all squares known to be occupied, i.e. 100% chance of being occupied.

A [bitboard](https://www.chessprogramming.org/Bitboards) is a 64-bit integer, where each bit corresponds to a square on the chess board. The bitboard is encoded in little endian form, with the least significant bit corresponding to a1, and increasing along rows (b1, c1, ...) up to h8 in the most significant bit.


##init(init_basis_state) : { probabilities, empty_bitboard, full_bitboard }
The init function is used to initialize a quantum state to some classical starting position. It takes a single argument, init_basis_state, and returns a json object with three fields: probabilities, empty_bitboard, and full_bitboard.
* init_basis_state: 
A bitboard which represents the initial classical state of the board, i.e. which squares have a piece on them.

Here we define an implementation of init that prints out the probability distribution of the initialized board, and returns the appropriate json.

In [ ]:
def init(board, init_basis_state):
  board.with_state(init_basis_state)
  probs = board.get_probability_distribution()
  print_game(board, probs)

  return {
      'probabilities' : probs,
      'empty' : board.get_empty_squares_bitboard(),
      'full' : board.get_full_squares_bitboard()
  }

r = init(global_board, 0xFFFF00000000FFFF)

        

##do_move( move ) : { probabilities, empty_bitboard, full_bitboard }
The do_move function is used to apply a specific unitary to the qubits which corresponding to the squares involved in the move. It takes a single argument, move, which is a json object with the following fields:
* square1: integer index of the first square.
* square2: integer index of the second square.
* square3: integer index of the third square, only used for split and merge moves.
* type: enumerated type of move
```
    NULL_TYPE = 0, UNSPECIFIED_STANDARD = 1, JUMP = 2, SLIDE = 3,
    SPLIT_JUMP = 4, SPLIT_SLIDE = 5, MERGE_JUMP = 6, MERGE_SLIDE = 7,
    PAWN_STEP = 8, PAWN_TWO_STEP = 9, PAWN_CAPTURE = 10, PAWN_EP = 11,
    KS_CASTLE = 12, QS_CASTLE = 13 
```
* variant: enumerated variant of move
```
    UNSPECIFIED = 0, BASIC = 1, EXCLUDED = 2, CAPTURE = 3
```

Here we define some helper functions to create Moves, which are used to apply specific unitaries to the qubits represented in the CirqBoard, and an implementation of do_move that will print the probability distribution for the pieces after the move.

In [ ]:
from recirq.quantum_chess.move import Move
from recirq.quantum_chess.enums import MoveType, MoveVariant

# Helper function for creating a split move from json values
def get_split_move(move_json):
  return Move(
    move_json['square1'],
    move_json['square2'],
    target2 = move_json['square3'],
    move_type = MoveType(move_json['type']),
    move_variant = MoveVariant(move_json['variant'])
    )

# Helper function for creating a merge move from json values
def get_merge_move(move_json):
  return Move(
    move_json['square1'],
    move_json['square3'],
    source2 = move_json['square2'],
    move_type = MoveType(move_json['type']),
    move_variant = MoveVariant(move_json['variant'])
    )

# Helper function for creating a standard move from json values
def get_standard_move(move_json):
  return Move(
    move_json['square1'],
    move_json['square2'],
    move_type = MoveType(move_json['type']),
    move_variant = MoveVariant(move_json['variant'])
    )
    
def do_move(board, move):
  board.clear_debug_log()
  r = board.do_move(move)
  probs = board.get_probability_distribution()
  print_game(board, probs)

  return {
      'result' : r,
      'probabilities' : probs,
      'empty' : board.get_empty_squares_bitboard(),
      'full' : board.get_full_squares_bitboard()
  }

move_json = {'square1' : 'b1', 'square2' : 'a3', 'square3' : 'c3', 'type' : MoveType.SPLIT_JUMP, 'variant': MoveVariant.BASIC}
split_b1_a3_c3 = get_split_move(move_json)

r = init(global_board, 0xFFFF00000000FFFF)
r = do_move(global_board, split_b1_a3_c3)
    
        

Notice, the circuit for te move is printed as well. This is made available in the board debug information. We can also see what happens when we use a noisy simulator for the board, and no error mitigation.

In [ ]:
from recirq.quantum_chess.enums import ErrorMitigation
from cirq import DensityMatrixSimulator, google
from cirq.contrib.noise_models import DepolarizingNoiseModel

NOISY_SAMPLER = DensityMatrixSimulator(noise= DepolarizingNoiseModel(
    depol_prob=0.004))

noisy_board = CirqBoard(0,
                 sampler=NOISY_SAMPLER,
                 device=google.Sycamore,
                 error_mitigation= ErrorMitigation.Correct,
                 noise_mitigation=0.05)

r = init(noisy_board, 0xFFFF00000000FFFF)
r = do_move(noisy_board, split_b1_a3_c3)

You may notice that the circuit run discarded some of the returned samples due to error mitigation and post selection.

##undo_last_move( ) : { probabilities, empty_bitboard, full_bitboard }
The undo_last_move function is used revert the quantum state to a state immediately before the last move that was executed. It takes no arguments, and returns the same json object as the previous endpoints.

In [ ]:
def undo_last_move(board):
  board.clear_debug_log()

  r = board.undo_last_move()
  probs = board.get_probability_distribution()
  print_game(board, probs)

  return {
      'result' : r,
      'probabilities' : probs,
      'empty' : board.get_empty_squares_bitboard(),
      'full' : board.get_full_squares_bitboard()
  }

r = init(global_board, 0xFFFF00000000FFFF)
r = do_move(global_board, split_b1_a3_c3)
r = undo_last_move(global_board)

With the functionality in place, we can define our endpoints and run the server. We will use the [flask_restful](https://flask-restful.readthedocs.io/en/latest/) framework to create a simple server that implements these enpoints. Flask-restful allows us to encapsulate the functionality we want in classes that inherit from Resource. We will need to install flask-ngrok to give our server an accessible url.

In [ ]:
!pip install flask -q
!pip install flask_restful -q
!pip install flask-ngrok -q

In [ ]:
from flask import Flask, request, jsonify
from flask_restful import Resource, Api
from flask_ngrok import run_with_ngrok

class Init(Resource):
  def get(self):
    return {'about': 'Init'}

  def post(self):
    print(request.get_json())
    n = request.get_json()['init_basis_state']
    global_board.clear_debug_log()
    return init(global_board,int(n))

    
class DoMove(Resource):
  def post(self):
    move_json = request.get_json()
    t = MoveType(move_json['type'])
    # We need to convert square indices to square names.
    move_json['square1'] = bit_to_square(move_json['square1'])
    move_json['square2'] = bit_to_square(move_json['square2'])
    move_json['square3'] = bit_to_square(move_json['square3'])

    if t == MoveType.SPLIT_SLIDE or t == MoveType.SPLIT_JUMP:
      return do_move(global_board, get_split_move(move_json))
    elif t == MoveType.MERGE_JUMP or t == MoveType.MERGE_SLIDE:
      return do_move(global_board, get_merge_move(move_json))
    else:
      return do_move(global_board, get_standard_move(move_json))

class UndoLastMove(Resource):
    def post(self):
        return undo_last_move(global_board)

app = Flask(__name__)
run_with_ngrok(app)

api = Api(app)

api.add_resource(Init, '/quantumboard/init')
api.add_resource(DoMove, '/quantumboard/do_move')
api.add_resource(UndoLastMove, '/quantumboard/undo_last_move')

@app.route("/")
def home():
    return "<h1>Running Flask on Google Colab!</h1>"

if __name__ == '__main__':
    app.run()

The server should now be running, and can be tested with the [Quantum Chess Client](./Quantum_Chess_Client.ipynb) notebook!